In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp
import time

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28,28)) / 255.
  return images
  # return np.where(images > .5, 1.0, 0.0).astype('float32')

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

train_labels = np.expand_dims(train_labels,axis=-1)
test_labels = np.expand_dims(test_labels,axis=-1)
train_size = 60000
batch_size = 200
test_size = 10000

# train_labels_binary = train_labels
# test_labels_binary = test_labels
# train_labels_binary = (train_labels == 5).astype(np.int)
# test_labels_binary = (test_labels == 5).astype(np.int)

train_images = tf.expand_dims(train_images, axis = -1)
test_images = tf.expand_dims(test_images, axis = -1)

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((train_images, train_labels))
                 .shuffle(train_size,reshuffle_each_iteration=True).batch(batch_size,drop_remainder=True))
test_dataset = (tf.data.Dataset.from_tensor_slices((test_images,test_labels))
                .shuffle(test_size).batch(batch_size,drop_remainder=True))

In [ ]:
train_labels.shape, train_images.shape

((60000, 1), TensorShape([60000, 28, 28, 1]))

In [ ]:

# train_images = tf.expand_dims(train_images, axis = -1)
# test_images = tf.expand_dims(test_images, axis = -1)

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:

class CustomDropout(tf.keras.layers.Layer):
    def __init__(self, rate, input_dim, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = 1-rate
        # self.hiddem_dim = hidden_dim
        self.input_dim = input_dim
        self.mask_w = self.add_weight(shape=(self.input_dim,n_decision_makers), trainable=True)
        self.mask_b = self.add_weight(shape=(n_decision_makers,), initializer="zeros",trainable=True)
        # self.mask = tf.ones(shape = (batch_size,n_decision_makers*10))
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=False) ## units=n_decision_maker
   
    def call(self, inputs, label, training=None):
        # inputs(batchsize, n_decision_maker, 10)
        if training:
          # print([label])
          scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
          # scce = tf.keras.losses.SparseCategoricalCrossentropy()
          # print(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]))
          # print(scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs))
          # tf.tile(, [1,n_decision_makers])
          # loss = tf.math.reduce_sum(scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs),axis=0) ## (n_decision_maker)
          loss = scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs)
          # loss = tf.math.reduce_sum(loss,axis=0)
          # print(tfp.stats.percentile(loss, q=self.rate*100))
          threshold = tfp.stats.percentile(loss, q=self.rate*100)
          dropout_mask = (loss<=threshold) ## <= 1-rate keep the best 10%
          # print(np.where(dropout_mask))
          # mask = tf.tile(tf.expand_dims(tf.expand_dims(dropout_mask, axis=-1),axis=0), [batch_size,1,10])
          mask = tf.tile(tf.expand_dims(dropout_mask, axis=-1), [1,1,10])
          # self.mask += tf.cast(mask,'float32')
          # print(dropout_mask, mask)
          # print(dropout_mask.shape, mask.shape)
          # dropout_mask = np.random.binomial([np.ones((self.input_dim,self.hidden_num))],1-self.rate)[0] * (1.0/(1-self.rate))
          # return self.activation(tf.matmul(inputs, tf.multiply(dropout_mask, self.w) )+ self.b)
      # return self.activation(tf.matmul(inputs, self.w) + self.b)
          # mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)

          mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.tile(mask_pred, [1,10])
          return tf.multiply(tf.keras.layers.Reshape((n_decision_makers,10))(mask_pred), inputs), tf.cast(mask,'float32'), mask_pred
        # print(tf.cast(self.mask,'float32'))
        else:
          # mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.tile(mask_pred, [1,10])
          return tf.multiply(tf.keras.layers.Reshape((n_decision_makers,10))(mask_pred),inputs),tf.ones(shape = (batch_size,n_decision_makers,10)),mask_pred ## reshape self.mask

In [ ]:

class CustomDropout(tf.keras.layers.Layer):
    def __init__(self, rate, input_dim, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = 1-rate
        # self.hiddem_dim = hidden_dim
        self.input_dim = input_dim
        self.mask_w = self.add_weight(shape=(self.input_dim,n_decision_makers), trainable=True)
        self.mask_b = self.add_weight(shape=(n_decision_makers,), initializer="zeros",trainable=True)
        # self.mask = tf.ones(shape = (batch_size,n_decision_makers*10))
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=False) ## units=n_decision_maker
   
    def call(self, inputs, label, training=None):
        # inputs(batchsize, n_decision_maker, 10)
        if training:
          # print([label])
          scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
          # scce = tf.keras.losses.SparseCategoricalCrossentropy()
          # print(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]))
          # print(scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs))
          # tf.tile(, [1,n_decision_makers])
          # loss = tf.math.reduce_sum(scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs),axis=0) ## (n_decision_maker)
          loss = scce(tf.tile(tf.transpose([label],perm = [1,2,0]),[1,n_decision_makers,1]),inputs)
          # loss = tf.math.reduce_sum(loss,axis=0)
          # print(tfp.stats.percentile(loss, q=self.rate*100))
          threshold = tfp.stats.percentile(loss, q=self.rate*100)
          dropout_mask = (loss<=threshold) ## <= 1-rate keep the best 10%
          # print(np.where(dropout_mask))
          # mask = tf.tile(tf.expand_dims(tf.expand_dims(dropout_mask, axis=-1),axis=0), [batch_size,1,10])
          mask = tf.tile(tf.expand_dims(dropout_mask, axis=-1), [1,1,10])
          # self.mask += tf.cast(mask,'float32')
          # print(dropout_mask, mask)
          # print(dropout_mask.shape, mask.shape)
          # dropout_mask = np.random.binomial([np.ones((self.input_dim,self.hidden_num))],1-self.rate)[0] * (1.0/(1-self.rate))
          # return self.activation(tf.matmul(inputs, tf.multiply(dropout_mask, self.w) )+ self.b)
      # return self.activation(tf.matmul(inputs, self.w) + self.b)
          # mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)

          mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.tile(mask_pred, [1,10])
          return tf.multiply(tf.keras.layers.Reshape((n_decision_makers,10))(mask_pred), inputs), tf.cast(mask,'float32'), mask_pred
        # print(tf.cast(self.mask,'float32'))
        else:
          # mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.nn.sigmoid(tf.matmul(tf.keras.layers.Flatten()(inputs), self.mask_w)+self.mask_b)
          mask_pred = tf.tile(mask_pred, [1,10])
          return tf.multiply(tf.keras.layers.Reshape((n_decision_makers,10))(mask_pred),inputs),tf.ones(shape = (batch_size,n_decision_makers,10)),mask_pred ## reshape self.mask

In [ ]:
n_decision_makers = 4  #100
class MyModel(tf.keras.Model):
    def __init__(self,**kwargs):
      super(MyModel,self).__init__(**kwargs)
      # self.inp = tf.keras.layers.Input(shape=input_shape)
      # self.flat = tf.keras.layers.Flatten()
      # self.flat1 = tf.keras.layers.Flatten()
      # self.flat2 = tf.keras.layers.Flatten()
      # self.reshape1 = tf.keras.layers.Reshape((n_decision_makers,10)) 
      # self.dropout1 = CustomDropout(0.2, n_decision_makers*10 )
      # self.dense1 = tf.keras.layers.Dense(n_decision_makers*10,activation=tf.nn.relu)
      # self.dense2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

      self.flat1 = tf.keras.layers.Flatten()
      self.flat2 = tf.keras.layers.Flatten()
      self.flat3 = tf.keras.layers.Flatten()
      self.flat4 = tf.keras.layers.Flatten()
      self.flat5 = tf.keras.layers.Flatten()
      self.flat6 = tf.keras.layers.Flatten()
      # self.flat7 = tf.keras.layers.Flatten()
      self.reshape1 = tf.keras.layers.Reshape((n_decision_makers,10))
      self.reshape2 = tf.keras.layers.Reshape((n_decision_makers,10))
      # self.reshape3 = tf.keras.layers.Reshape((n_decision_makers,10))


      
      self.dropout1 = CustomDropout(0.7,n_decision_makers*10)
      # self.dropout2 = CustomDropout(0.1,n_decision_makers*10)
      # self.dropout3 = CustomDropout(0)

      
      # self.dropout1 = tf.keras.layers.Dropout(0.2)
      # self.dropout2 = tf.keras.layers.Dropout(0.4)
      # self.dropout3 = tf.keras.layers.Dropout(0.2)

      self.dropout4 = tf.keras.layers.Dropout(0.2)
      self.dropout5 = tf.keras.layers.Dropout(0.2)
      # self.dropout6 = tf.keras.layers.Dropout(0.4)

      self.pool1 = tf.keras.layers.MaxPooling2D((2, 2))
      self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))
      # self.pool3 = tf.keras.layers.MaxPooling2D((2, 2))
      # self.pool4 = tf.keras.layers.MaxPooling2D((2, 2))

      # self.concat1 = tf.keras.layers.Concatenate()
      # self.concat2 = tf.keras.layers.Concatenate()

      # self.avgpool1 = tf.keras.layers.GlobalAveragePooling2D()
      # self.avgpool2 = tf.keras.layers.GlobalAveragePooling2D()
      # self.avgpool3 = tf.keras.layers.GlobalAveragePooling2D()
      # self.avgpool4 = tf.keras.layers.GlobalAveragePooling2D()

      self.conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      self.conv11 = tf.keras.layers.Conv2D(128, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      self.dense1 = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
      self.batchnorm1 = tf.keras.layers.BatchNormalization()

      self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      self.conv22 = tf.keras.layers.Conv2D(64, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      self.dense2 = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
      self.batchnorm2 = tf.keras.layers.BatchNormalization()

      # self.conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      # self.conv33 = tf.keras.layers.Conv2D(256, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      # self.conv34 = tf.keras.layers.Conv2D(256, 3, activation='relu',padding='same',kernel_regularizer=tf.keras.regularizers.l1(l=0.01),kernel_initializer='he_uniform',)
      # self.dense3 = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
      # self.batchnorm3 = tf.keras.layers.BatchNormalization()

      # self.conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu',padding='same',kernel_initializer='he_uniform',)
      # self.conv41 = tf.keras.layers.Conv2D(512, 3, activation='relu',padding='same',kernel_initializer='he_uniform',)
      # self.conv42 = tf.keras.layers.Conv2D(512, 3, activation='relu',padding='same',kernel_initializer='he_uniform',)
      # self.dense4 = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
      # self.batchnorm4 = tf.keras.layers.BatchNormalization()
      
      # self.dense4 = tf.keras.layers.Dense(n_decision_makers*10,activation=tf.nn.relu)
      self.dense5 = tf.keras.layers.Dense(n_decision_makers*10,activation=tf.nn.relu)
      # self.dense6 = tf.keras.layers.Dense(n_decision_makers*10,activation=tf.nn.relu)

      self.dense7 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

    def call(self, input):
      # (inputs,label) = input
      
      [input, label] = input
      hidden_conv1 = self.dropout4(self.batchnorm1(self.pool1(self.conv1(self.conv11(input)))))
      hidden_conv1_reshape = self.flat4(hidden_conv1)
      # print(hidden_conv1.shape)
      #hidden_conv1_out = self.dense1(hidden_conv1_reshape)

      hidden_conv2 = self.dropout5(self.batchnorm2(self.pool2(self.conv2(self.conv22(hidden_conv1)))))
      hidden_conv2_reshape = self.flat5(hidden_conv2)
      # hidden_conv2_reshape = self.avgpool1(hidden_conv2)
      #hidden_conv2_out = self.dense2(hidden_conv2_reshape)

      # hidden_conv3 = self.dropout6(self.batchnorm3(self.pool3(self.conv3(hidden_conv2))))
      # hidden_conv3_reshape = self.flat6(hidden_conv3)
      # hidden_conv3_out = self.dense3(hidden_conv3_reshape)

      # hidden_conv4 = self.dropout7(self.batchnorm4(self.pool4(self.conv4(self.conv41(self.conv42(hidden_conv3))))))
      # hidden_conv4_reshape = self.flat7(hidden_conv4)
      # hidden_conv4_out = self.dense4(hidden_conv4_reshape)

      # x = self.flat(input)
      # print(x.shape)
      
      hidden1 = self.dense5(hidden_conv2_reshape)

    # def call(self, input):
    #   [input, label] = input
    #   x = self.flat(input)
    #   # print(label.shape)
      
    #   hidden1 = self.dense1(x)

      hidden1_reshape = self.reshape1(hidden1)
      hidden1_softmax = tf.nn.softmax(hidden1_reshape)
      hidden1_out,hidden1_true_mask,hidden1_pred_mask = self.dropout1(hidden1_softmax,label)
      # hidden1_drop = tf.multiply(hidden1_true_mask, hidden1_reshape)

      # x_reshape = tf.squeeze(tf.image.resize(tf.expand_dims(x,-1),[n_decision_makers*10,1]))
      # hidden2 = self.dense6(self.flat1(hidden1_drop))
      # hidden2_reshape = self.reshape2(self.dropout2(hidden2))
      # hidden2_reshape = self.batchnorm2(hidden2_reshape)
      # hidden2_drop = self.dropout2(hidden2_reshape,label)
      # hidden2_drop = self.dropout2(hidden2_reshape)
      # hidden2_out = tf.nn.softmax(hidden2_reshape,axis=-1)
      # hidden2_out = tf.nn.softmax(hidden2_reshape,axis=-1)
      

      # hidden3 = self.dense6(self.flat2(hidden2_drop)) 
      # hidden3_reshape = self.reshape3(hidden3)
      # hidden3_drop = self.dropout3(hidden3_reshape,label)
      # hidden3_out = tf.nn.softmax(hidden3_drop,axis=-1)

      # hidden2 = self.dropout2(hidden2)
      outputs = self.dense7(self.flat1(hidden1_out)) #leader outputs
      #outputs = self.dense7(hidden_conv2_out)# BP
      # print(outputs.shape)
      # return hidden1, hidden2,outputs
      
      #return self.flat2(hidden1_true_mask), hidden1_pred_mask, hidden_conv1_out, hidden_conv2_out, hidden1_out, outputs
      return outputs

In [ ]:
# input_shape=(None, 28, 28)
# label_shape=(None, 1)
model = MyModel()
model([tf.zeros((batch_size, 28, 28, 1)),tf.zeros((batch_size, 1))])
# model.build([input_shape, label_shape])
# model.build_graph().summary()
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 flatten_2 (Flatten)         multiple                  0 (unused)
                                                                 
 flatten_3 (Flatten)         multiple                  0         
                                                                 
 flatten_4 (Flatten)         multiple                  0         
                                                                 
 flatten_5 (Flatten)         multiple                  0 (unused)
                                                                 
 reshape (Reshape)           multiple                  0  

In [ ]:
# acc=tf.keras.metrics.SparseCategoricalAccuracy()
model.compile(loss='sparse_categorical_crossentropy',
	 optimizer='adam',
	 metrics='acc')

model.fit([train_images, train_labels], train_labels,
	 validation_data=([test_images, np.zeros(test_labels.shape)],test_labels),
	 epochs=100,
	 batch_size=batch_size,
)

local_loss

In [ ]:
# from keras import backend as K


# train_loss, test_loss = tf.keras.metrics.Mean(),tf.keras.metrics.Mean()
# mask_loss = tf.keras.metrics.Mean()
# train_acc = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
# test_acc = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')


# initial_learning_rate = 1e-4
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True)
# # optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule,clipnorm = 1)
# # # optimizer_mask = tf.keras.optimizers.Adam()

# optimizer_mask = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 5e-4)


# scce = tf.keras.losses.SparseCategoricalCrossentropy()
# cce = tf.keras.losses.BinaryCrossentropy()

# def compute_loss(hidden1_true_mask, hidden1_pred_mask, conv1, conv2, hidden1, y, output):
#   loss_local_conv1 = scce(y,conv1)
#   loss_local_conv2 = scce(y,conv2)
#   # loss_local_conv3 = scce(y,hidden_conv3)cust
#   # loss_local_conv4 = scce(y,hidden_conv4)

#   # loss_local_hidden1 = scce(tf.tile(tf.transpose([y]), [1,n_decision_makers]),hidden1)
#   # loss_local_hidden2 = scce(tf.tile(tf.transpose([y]), [1,n_decision_makers]),hidden2)

#   loss_local_hidden1 = scce(tf.tile(y, [1,n_decision_makers]),hidden1)
#   # loss_local_hidden2 = scce(tf.tile(y, [1,n_decision_makers]),hidden2)

#   # loss_local_hidden1 = scce(tf.tile(tf.transpose([y]), [1,n_decision_makers]),hidden1)

#   # loss_local_hidden1 = scce(tf.tile(y, [1,n_decision_makers]),hidden1)
#   # print(loss_local)
#   loss = scce(y,output)

#   loss_mask = cce(hidden1_true_mask, hidden1_pred_mask)

#   return loss, loss_mask, loss_local_conv1, loss_local_conv2, loss_local_hidden1

# def compute_loss_mask(hidden1_true_mask, hidden1_pred_mask):
#   loss_mask = cce(hidden1_true_mask, hidden1_pred_mask)

#   return loss_mask

# def compute_acc(model, x, y):
#   # m = tf.keras.metrics.BinaryAccuracy()
#   _,_,_,_,_,output = model([x,y])
#   # acc = m(y, output)
#   acc = tf.keras.metrics.sparse_categorical_accuracy(y, output)
#   return acc

# # @tf.function
# def train_step(model, x, y, optimizer):
#   with tf.GradientTape(persistent =True) as tape:
#     # output = model([x,y], training=True)
#     # loss = scce(y, output)
#     hidden1_true_mask, hidden1_pred_mask, conv1, conv2, hidden1, output = model([x,y], training=True)
#     loss, loss_mask, loss_local_conv1, loss_local_conv2, loss_local_hidden1 = compute_loss(hidden1_true_mask, hidden1_pred_mask, conv1, conv2, hidden1, y, output)

#   # gradients_global = tape.gradient(loss, model.trainable_variables)
#   # optimizer.apply_gradients(zip(gradients_global, model.trainable_variables))
#   gradients_global = tape.gradient(loss, model.layers[-1].trainable_variables)
#   optimizer.apply_gradients(zip(gradients_global, model.layers[-1].trainable_variables))

#   gradients_local = tape.gradient(loss_local_hidden1, model.layers[-2].trainable_variables)
#   optimizer.apply_gradients(zip(gradients_local, model.layers[-2].trainable_variables))

#   gradients_local = tape.gradient(loss_local_conv2,model.trainable_variables[10:17])
#   optimizer.apply_gradients(zip(gradients_local, model.trainable_variables[10:17]))

#   gradients_local = tape.gradient(loss_local_conv1,model.trainable_variables[2:10])
#   optimizer.apply_gradients(zip(gradients_local, model.trainable_variables[2:10]))

#   gradients_local = tape.gradient(loss_mask, model.layers[-15].trainable_variables)
#   optimizer_mask.apply_gradients(zip(gradients_local, model.layers[-15].trainable_variables))

#   train_acc(y,output)
#   train_loss(loss)

#   for i in range(3):
#     with tf.GradientTape(persistent =True) as tape:
#       hidden1_true_mask, hidden1_pred_mask, _ ,_,_,_= model([x,y], training=True)
#       loss_mask = compute_loss_mask(hidden1_true_mask, hidden1_pred_mask)
#     gradients_local = tape.gradient(loss_mask, model.layers[-15].trainable_variables)
#     optimizer_mask.apply_gradients(zip(gradients_local, model.layers[-15].trainable_variables))

#   mask_loss(loss_mask)
#   # return loss

# def test_step(model, x, y):
#   _,_,_,_,_,output = model([x,y])
#   scce = tf.keras.losses.SparseCategoricalCrossentropy()
#   loss = scce(y, output)

#   test_loss(loss)
#   test_acc(y, output)

In [ ]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [ ]:
# import tensorflow as tf
# import datetime

# # Clear any logs from previous runs
# !rm -rf ./logs/

In [ ]:
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# train_log_dir = 'logs/gradient_tape/' + current_time + str(n_decision_makers)+'dropout/train'
# test_log_dir = 'logs/gradient_tape/' + current_time + str(n_decision_makers)+'dropout/test'
# train_summary_writer = tf.summary.create_file_writer(train_log_dir)
# test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
# EPOCHS = 50
# from tqdm.notebook import tqdm

# import time

# for epoch in range(0, 0+EPOCHS):
#   start=time.time()
#   for i, (train_x, train_y) in enumerate(tqdm(train_dataset)):
#     train_step(model, train_x, train_y, optimizer)
#     # print(model.layers[-3].trainable_variables)
#     #   break
#   # break
#   with train_summary_writer.as_default():
#     tf.summary.scalar('loss', train_loss.result(), step=epoch)
#     tf.summary.scalar('loss', mask_loss.result(), step=epoch)
#     tf.summary.scalar('accuracy', train_acc.result(), step=epoch)
#   # break

#   for test_x, test_y in test_dataset:
#     test_step(model, test_x, test_y)
#   with test_summary_writer.as_default():
#     tf.summary.scalar('loss', test_loss.result(), step=epoch)
#     tf.summary.scalar('accuracy', test_acc.result(), step=epoch)

#   template = 'Epoch {}, Loss: {}, Accuracy: {}, MaskLoss: {}, Test Loss: {}, Test Accuracy: {}'
#   print(template.format(epoch+1,
#                          train_loss.result(), 
#                          train_acc.result()*100,
#                          mask_loss.result(), 
#                          test_loss.result(), 
#                          test_acc.result()*100))

#   # Reset metrics every epoch
#   train_loss.reset_states()
#   test_loss.reset_states()
#   mask_loss.reset_states()
#   train_acc.reset_states()
#   test_acc.reset_states()

#   # adv_test(model)
#   print("Time elapsed: ", time.time()-start)

  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1, Loss: 2.0551376342773438, Accuracy: 69.18333435058594, MaskLoss: 0.5811836123466492, Test Loss: 1.7700191736221313, Test Accuracy: 98.5
Time elapsed:  81.6336772441864


  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 2, Loss: 1.5729107856750488, Accuracy: 98.4366683959961, MaskLoss: 0.5530502200126648, Test Loss: 1.3804194927215576, Test Accuracy: 98.8499984741211
Time elapsed:  76.50606107711792


  0%|          | 0/300 [00:00<?, ?it/s]

...

  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 34, Loss: 0.5735808610916138, Accuracy: 71.15833282470703, MaskLoss: 0.3379290699958801, Test Loss: 0.5973612070083618, Test Accuracy: 70.91000366210938
Time elapsed:  76.74347567558289


  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 35, Loss: 0.5734826922416687, Accuracy: 71.1433334350586, MaskLoss: 0.34204110503196716, Test Loss: 0.598856508731842, Test Accuracy: 70.9000015258789
Time elapsed:  76.51881313323975


  0%|          | 0/300 [00:00<?, ?it/s]